# Here we'll train some models!

In [39]:

from __future__ import annotations
import pathlib, yaml
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import sys, pathlib

# --- project paths
PROJ = pathlib.Path().resolve().parent
CFG  = yaml.safe_load(open(PROJ / "conf" / "config.yaml"))
var_types = yaml.safe_load(open(PROJ / "conf" / "variable_types.yaml"))
RAW_DIR = PROJ / CFG["data"]["raw_dir"]
PROC_DIR = PROJ / CFG["data"]["processed_dir"]
numeric_cols = var_types["numeric_variables"]

if str(PROJ) not in sys.path:
    sys.path.insert(0, str(PROJ))

from pipeline.transformers import MedianImputer  # <-- change to what you need

In [ ]:
# First read in the file...
df = pd.read_parquet(PROC_DIR / "cbecs_2018_clean.parquet")

# Assign X, y 
X = df.drop('LOG_MFBTU', axis=1)
y = df['LOG_MFBTU']

# Split the data into validation, train, and test sets
# e.g., 80/10/10 split
test_size = 0.10
val_size  = 0.10
rand      = 42

# 1) Hold out test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=rand, shuffle=True
)

# 2) From the remainder, carve out validation
val_rel = val_size / (1.0 - test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_rel, random_state=rand, shuffle=True
    )

# Check and make sure the splits worked as expected...
df.shape
check_shape = X_train.shape[0]+X_test.shape[0]+X_val.shape[0]  == df.shape[0]

print(f"Split correctly: {check_shape}")

Split correctly: True


In [41]:
# # ---------- Feature engineering block  ----------
# linear_feat_engineer = Pipeline(steps = [

#     ("median_imputer", MedianImputer(RULES.get("impute_rules"))), # Impute median where necessary and add imputation flags because missing has meaning in this dataset
#     ('Auto_transform', AutoTransform(
#         scoring='r2',
#         cv=5,
#         n_jobs=-1,
#         num_cols = num_cols,
#         suffix_identity=False
#     )),
#     ('scaler', StandardScaler()),
#     ('select', SelectFromModel(LassoCV(cv=10, n_jobs=-1), prefit=False)),  # fast feature cut
#     ('model', LinearRegression())

# ])